# Wordle: https://www.powerlanguage.co.uk/wordle/
# Absurdle: https://qntm.org/files/absurdle/absurdle.html
# First, lets load up our wordlists.

In [ ]:
with open("wordle-allowed-guesses.txt") as f:
    allowedGuesses = [line.split("\n")[0] for line in f]
with open("wordle-answers-alphabetical.txt") as f:
    allowedAnswers = [line.split("\n")[0] for line in f]
allGuesses = allowedGuesses[:] + allowedAnswers[:]

# Next, lets define some helper functions to play a round, etc.

In [ ]:
def getOutcome(guess, secret):
    outcome = ['x' for _ in range(5)]
    secretD = {}
    for letter in secret:
        if letter in secretD:
            secretD[letter] += 1
        else:
            secretD[letter] = 1
    for i in range(5):
        if guess[i] == secret[i]:
            secretD[secret[i]] -= 1
            outcome[i] = 'g'
    for i in range(5):
        if outcome[i] != 'g' and guess[i] in secretD and secretD[guess[i]] > 0:
            outcome[i] = 'y'
            secretD[guess[i]] -= 1
    return "".join(outcome)

In [ ]:
def genAllOutcomes(i):
    if i <= 0:
        yield ''
    else:
        for outcome in genAllOutcomes(i-1):
            yield 'x' + outcome
            yield 'g' + outcome
            yield 'y' + outcome

allOutcomes = list(genAllOutcomes(5))

In [ ]:
def filterToBuckets(guess, dictionary):
    outcomes = {o: [] for o in allOutcomes}
    for word in dictionary:
        outcome = getOutcome(guess, word)
        outcomes[outcome].append(word)
    return outcomes

# This function pretty-prints everything.

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def fancyPrint(outcome, guess, end = "\n"):
    print("Outcome: ", end = "")
    for i in range(5):
        if outcome[i] == 'x':
            print(guess[i], end = "")
        elif outcome[i] == 'g':
            print(bcolors.OKGREEN + guess[i] + bcolors.ENDC, end = "")
        elif outcome[i] == 'y':
            print(bcolors.WARNING + guess[i] + bcolors.ENDC, end = "")
    print("", end=end)


# Let's simulate a round of Wordle.

In [ ]:
import random

def interactWordle():
    secretWord = random.choice(allowedAnswers)
    allLetters = list("etaoinshrdlcumwfgypbvkjxqz") # by frequency
    while True:
        guess = input("Enter your guess:")
        if guess == "":
            break
        else:
            if guess in allGuesses:
                outcome = getOutcome(guess, secretWord)
                fancyPrint(outcome, guess)
                if outcome == "ggggg":
                    print("You win!")
                else:
                    for letter in guess:
                        try:
                            i = allLetters.index(letter)
                            allLetters.pop(i)
                        except ValueError:
                            pass
                    print("Unused letters:", "".join(allLetters))
                    print("***************")
            else:
                print("Guess not in dictionary.")

# Let's play an example game of Wordle (uncomment this function call).

In [ ]:
# interactWordle()

# Let's try minmax to find the best Wordle guesses.

In [ ]:
def iterateWordle(dictionary):
    minMaxBucket = len(dictionary)
    if minMaxBucket == 1:
        return dictionary[0], {"ggggg": dictionary[0]}
    else:
        bestGuess = ""
        bestBuckets = dict()
        for guess in allGuesses:
            buckets = filterToBuckets(guess, dictionary)
            maxBucketSize = max([len(buckets[t]) for t in buckets])
            if maxBucketSize < minMaxBucket:
                minMaxBucket = maxBucketSize
                bestGuess = guess
                bestBuckets = buckets
        return bestGuess, bestBuckets

In [ ]:
def solveWordle(dictionary = None):
    if dictionary is None:
        dictionary = allowedAnswers
        first = True
    else:
        first = False
    while True:
        if first:
            bestGuess = "aesir"
            bestBuckets = filterToBuckets(bestGuess, dictionary)
            first = False
        else:
            bestGuess, bestBuckets = iterateWordle(dictionary)
        outcome = input(f"Outcome when you try \"{bestGuess}\" (combination of x, y, and g):")
        if not outcome or outcome == "ggggg":
            break
        fancyPrint(outcome, bestGuess)
        dictionary = bestBuckets[outcome]

Uncomment this for interactive Wordle solver.

In [ ]:
# solveWordle()

# A solver for Wordle that picks up mid game.
Uncomment to run.

In [ ]:
# guessOutcomePairs = [('raise', 'yxxxx'), ('clout', 'xxyyx'), ('rough', 'yyyxy')]

# dictionary = allowedAnswers
# for (g, o) in guessOutcomePairs:
#     dictionary = filterToBuckets(g, dictionary)[o]
# solveWordle(dictionary)

# Now, let's check out Absurdle, the adversarial variant of Wordle. First some helper functions to generate outcomes and filter a word list by guess.

In [ ]:
def outcomeIsWorse(outcome, worstOutcome):
    if outcome.count('g') < worstOutcome.count('g'):
        return True
    elif outcome.count('g') == worstOutcome.count('g'):
        return outcome.count('y') < worstOutcome.count('y')
    else:
        return False

In [ ]:
def filterOnGuess(guess, currentWords):
    """
    guess: one of the allowed guesses
    currentWords: list of words that are still the possible secrets

    output: returned as (outcome, remainingWords)
    outcome: 5 character long string of ('x', 'g', 'y')
    remainingWords: list of remaining words
    """
    outcomes = {o: [] for o in allOutcomes}
    for word in currentWords:
        outcome = getOutcome(guess, word)
        outcomes[outcome].append(word)
    worstOutcome = ''
    mostSecrets = 0
    for outcome in outcomes:
        thisSecrets = len(outcomes[outcome])
        if thisSecrets > mostSecrets or thisSecrets == mostSecrets and outcomeIsWorse(outcome, worstOutcome):
            mostSecrets = thisSecrets
            worstOutcome = outcome
    return worstOutcome, outcomes[worstOutcome], outcomes

# Simulate a round of Absurdle.

In [ ]:
def interactAbsurdle():
    currentWords = allGuesses[:]
    allLetters = list("etaoinshrdlcumwfgypbvkjxqz") # by frequency
    while True:
        print("***************")
        guess = input("Enter your guess:")
        if guess == "":
            break
        else:
            if guess in allGuesses:
                outcome, currentWords, _ = filterOnGuess(guess, currentWords)
                fancyPrint(outcome, guess)
                if outcome == "ggggg":
                    print("You win!")
                else:
                    for letter in guess:
                        try:
                            i = allLetters.index(letter)
                            allLetters.pop(i)
                        except ValueError:
                            pass
                    print("Unused letters:", "".join(allLetters))
                    print("Number of words remaining:", len(currentWords))
                    print("***************")
            else:
                print("Guess not in dictionary.")

# Let's play an example game of Absurdle (uncomment this function call).

In [ ]:
# interactAbsurdle()

# Now, let's try to greedily find the best guesses.

In [ ]:
def iterateAbsurdle(dictionary):
    lastMin = len(dictionary)
    bestGuess = None
    bestOutcome = 'ggggg'
    bestCurrentWords = []
    for guess in allGuesses:
        outcome, currentWords, _ = filterOnGuess(guess, dictionary)
        if len(currentWords) <= lastMin:
            bestGuess = guess
            bestCurrentWords = currentWords
            bestOutcome = outcome
            lastMin = len(currentWords)
    fancyPrint(bestOutcome, bestGuess)
    if len(currentWords) == 1:
        print("***************************************************************")
        print("Done. ", end = "")
        fancyPrint('ggggg', currentWords[0])
        return []
    else:
        print("***************************************************************")
        return bestCurrentWords

# This cell shows that the solution obtained from greedy maximization is raise, duply, witch, zonal, uncut.
This cell is commented to avoid repeated execution. Uncomment the code before running it.

In [ ]:
# currentWords = allowedAnswers[:]
# while currentWords:
#     currentWords = iterateAbsurdle(currentWords)

# This cell tries to find a greedily optimized solution after a particular set of guesses.
This cell is commented to avoid repeated execution. Uncomment the code before running it.

In [ ]:
# alreadyGuessed = ["raise", "witch", "duply"]

# currentWords = allowedAnswers[:]
# for guess in alreadyGuessed:
#     outcome, currentWords, _ = filterOnGuess(guess, currentWords)
#     fancyPrint(outcome, guess)
# print("***************************************************************")
# while currentWords:
#     currentWords = iterateAbsurdle(currentWords)

# Now, let's try to solve the game in Challenge mode (where the final guess has to be a specific word)

In [ ]:
def iterateAbsurdleChallenge(target, dictionary, prevGuesses):
    lastMin = len(dictionary)
    bestGuess = None
    bestOutcome = 'ggggg'
    bestCurrentWords = []
    for guess in allGuesses:
        if guess not in prevGuesses:
            outcome, currentWords, buckets = filterOnGuess(guess, dictionary)
            if target in currentWords:
                # the largest bucket must contain the target word and must be one larger than the rest
                if len(currentWords) < lastMin:
                    lens = [len(buckets[t]) for t in buckets]
                    lens.sort(reverse=True)
                    if lens[0] > lens[1]:
                        bestGuess = guess
                        bestOutcome = outcome
                        bestCurrentWords = currentWords
                        lastMin = len(bestCurrentWords)
    fancyPrint(bestOutcome, bestGuess)
    if len(bestCurrentWords) == 1:
        print("***************************************************************")
        print("Done. ", end = "")
        fancyPrint('ggggg', bestCurrentWords[0])
        return bestCurrentWords[0], bestCurrentWords
    else:
        print("***************************************************************")
        return bestGuess, bestCurrentWords

Code is commented to stop it from executing.

In [ ]:
# targetSecret = "smear"

# currentWords = allowedAnswers[:]
# prevGuesses = []
# while len(currentWords) > 2:
#     bg, currentWords = iterateAbsurdleChallenge(targetSecret, currentWords, prevGuesses)
#     prevGuesses.append(bg)
# for guess in allGuesses:
#     if guess not in prevGuesses:
#         outcome, finalWords, buckets = filterOnGuess(guess, currentWords)
#         if len(finalWords) == 1 and finalWords[0] == targetSecret:
#             prevGuesses.append(guess)
#             fancyPrint(outcome, guess)
#             print("***************************************************************")
#             break
# fancyPrint("ggggg", targetSecret)